In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import re
from google.cloud import bigquery
import plotly.graph_objects as go


from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t

client = bigquery.Client()

In [ ]:
# Summarise the data by mcg
UK_index_merchant_channel = '''SELECT time_period_value, cardholders
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Month'
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM'
and mcg = 'All'
and merchant_channel = 'All'
GROUP BY time_period_value, cardholders
ORDER BY time_period_value, cardholders DESC'''

UK_index_merchant_channel = bq.read_bq_table_sql(client, UK_index_merchant_channel)

UK_index_merchant_channel.head()

In [ ]:
UK_index_merchant_channel.to_csv('UK_index_merchant_channel.csv')

In [ ]:
df_index = pd.read_excel("merchant channel 1.xlsx", engine="openpyxl")

# Convert time_period_value to datetime format
df_index["date"] = pd.to_datetime(df_index["time_period_value"].astype(str), format="%Y%m")

# Aggregate monthly spend for index file
monthly_index_spend = df_index.groupby("date")["spend"].sum().reset_index()

# Get the base value from January 2019
base_date = pd.to_datetime("201901", format="%Y%m")
base_value = monthly_index_spend.loc[monthly_index_spend["date"] == base_date, "spend"].values[0]

# Create monthly index (base = 100)
monthly_index_spend["monthly_index"] = (monthly_index_spend["spend"] / base_value) * 100

# Prepare main data for indexing
monthly_main_spend = df_main.groupby([df_main["date"], df_main["merchant_channel"]])["spend"].sum().reset_index()

# Merge with monthly index
merged_df = pd.merge(monthly_main_spend, monthly_index_spend[["date", "monthly_index"]], on="date", how="left")

# Extract data for 2019 to compute the base index for each merchant channel
merged_df["year"] = merged_df["date"].dt.year
base_2019 = merged_df[merged_df["year"] == 2019].groupby("merchant_channel")["spend"].mean()

# Normalize spend using 2019 average as base index (set to 100)
merged_df["indexed_spend"] = merged_df.apply(
    lambda row: (row["spend"] / base_2019[row["merchant_channel"]]) * 100, axis=1
)

# Apply monthly index from merchant channel 1.xlsx
merged_df["adjusted_indexed_spend"] = merged_df["indexed_spend"] * (merged_df["monthly_index"] / 100)

# Save the adjusted data to a CSV file
merged_df.to_csv("adjusted_monthly_spend_index.csv", index=False)
